# DATA20001 Deep Learning 2018 - Exercise 4

**Due Sunday December 02, before 23:59**


## Exercise 4.1 (3 points)

In this exercise we will implement a Convolutional Neural Network to perform sentiment analysis on the classic IMDB movie review dataset.

This is binary classification task, meaning we will be predicting either a positive (1) or negative (0) label. Your output should be a real value between 0 and 1. Use binary cross-entropy as your loss function.

Use 1D convolution and pooling layers. Use torch.nn.Embedding as the first layer to get the word embeddings for the input.

In [ ]:
import torch
from torchtext import datasets, vocab

if torch.cuda.is_available():
    print('Using GPU.')
    device = torch.device('cuda')
else:
    print('Using CPU.')
    device = torch.device('cpu')

glove = vocab.GloVe(name='6B', dim=50)
train_loader, test_loader = datasets.IMDB.iters(batch_size=4, device=device, vectors=glove)

## Exercise 4.2 (3 points)

For this exercise, do the same task as above, but replace the CNN with an LSTM. Everything else in your network should remain more or less the same.

Evaluate your results using the test data, comparing the two models.